In [1]:
import h5py
import pyccl
import scipy
import numpy

ModuleNotFoundError: No module named 'pyccl'

In [2]:
# Load in Data

H = 0.6736
NE = 3.046
NS = 0.9649
W = -1.0000
WA = 0.0000
AS = 2.1e-09
TAU = 0.0544
MNU = 0.0000
TCMB = 2.7250
SIGMA8 = 0.8233
OMEGAK = 0.0000
OMEGAB = 0.04930
OMEGAC = 0.26447
OMEGAM = 0.31377

COSMO_CCL = pyccl.core.Cosmology(
    h = H, 
    w0 = W, 
    wa = WA, 
    n_s = NS, 
    m_nu = MNU, 
    Neff = NE, 
    T_CMB = TCMB, 
    sigma8 = SIGMA8, 
    Omega_b = OMEGAB,
    Omega_k = OMEGAK, 
    Omega_c = OMEGAC,
    mass_function = 'tinker10', 
    emulator_neutrinos = 'strict', 
    halo_concentration = 'duffy2008', 
    matter_power_spectrum = 'halofit', 
    baryons_power_spectrum = 'nobaryons', 
    transfer_function = 'boltzmann_camb', 
    extra_parameters = {'camb': {'kmax':1.0e+4, 'lmax':5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

PATH = '/Volumes/ELEMENT/CTAM/'
DATA_PATH = PATH + 'DATA/'
FILE_PATH = PATH + 'FILE/'
PLOT_PATH = PATH + 'PLOT/'

BIN_SIZE = 4
GRID_SIZE = 72
LATENT_SIZE = 32
DATA_SIZE = 100000
COLOR = ['blue', 'green', 'orange', 'red']

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as FILE:
    
    Z_GRID = numpy.array(FILE['realizations/z'][:GRID_SIZE + 1], dtype = 'double')
    GRID = numpy.array(FILE['realizations/pdfs'][:, :BIN_SIZE, :GRID_SIZE + 1], dtype = 'double')

PSI_GRID = numpy.mean(GRID, axis = 0)
SIGMA_GRID = numpy.std(GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + Z_GRID))
CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + Z_GRID)) * H * 100000 / scipy.constants.c

ELL_SIZE = 100
ELL_DATA = numpy.logspace(1, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where = CHI_MESH > 0))

PMM_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):

    PMM_GRID[:,GRID_INDEX] = COSMO_CCL.nonlin_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

In [3]:
ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

1461.2041411753223 1497.7328186963734 50636.35304867648 49662.4601384986 0.36 0.37


In [4]:
def integral1(chi1, chi2, power1, power2, redshift1, redshift2):

    def formula(chi):
        
        result = ((chi2**2 - chi**2) / 2 / (chi2 - chi1) - chi * chi2 * numpy.log(chi2 / chi) / (chi2 - chi1))**2 
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)**2

        return result
    
    integral, error = scipy.integrate.quad_vec(f = formula, a = chi1, b = chi2)

    return integral

In [5]:
def coefficient1(chi1, chi2, power1, power2, redshift1, redshift2):

    a = 1 - chi1 / chi2
    b = 1 - power1 / power2
    c = 1 - (1 + redshift1) / (1 + redshift2)

    if a == 1:

        formula = (b * ( - 95795 + 2 * (87962 - 40703 * c) * c) + 14 * (7525 + c * ( - 13685 + 6283 * c))) / 5292000

    else: 

        formula = (1 / (5292000 * a**5)) * (
            + a * (
                + 245 * a**2 * (
                    + 10 * a * ( - 60 + a * ( - 750 + a * (1420 + 27 * a * ( - 25 + 4 * a)))) 
                    + (420 + a * (2010 + a * (1940 - 9 * a * (1005 + 4 * a * ( - 144 + 25 * a))))) * b
                ) + 14 * a * (
                    - 35 * a * ( - 420 + a * ( - 2010 + a * ( - 1940 + 9 * a * (1005 + 4 * a * ( - 144 + 25 * a))))) 
                    + 2 * ( - 5460 + a * ( - 15330 + a * ( - 14420 + 3 * a * ( - 4305 + 4 * a * (9534 + 125 * a * ( - 49 + 9 * a)))))) * b
                ) * c + (
                    + 14 * a * ( - 5460 + a * ( - 15330 + a * ( - 14420 + 3 * a * ( - 4305 + 4 * a * (9534 + 125 * a * ( - 49 + 9 * a)))))) 
                    + (59220 + a * (117810 + a * (107940 + a * (95655 + a * (85344 - 125 * a * (9968 + 27 * a * ( - 256 + 49 * a))))))) * b
                ) * c**2
            ) + 420 * ( - 1 + a)**2 * numpy.log(1 - a) * (
                + 35 * a**2 * (7 * b + a * ( - 10 + 74 * b + a * ( - 260 + a * (90 - 72 * b) + 171 * b))) 
                + 14 * a * (
                    + 5 * a * (7 + a * (74 + 9 * (19 - 8 * a) * a)) 
                    + 2 * ( - 13 + a * ( - 86 + 3 * a * ( - 63 + 2 * a * ( - 52 + 25 * a)))) * b
                ) * c + (
                    + 14 * a * ( - 13 + a * ( - 86 + 3 * a * ( - 63 + 2 * a * ( - 52 + 25 * a)))) 
                    + (141 + a * (702 + a * (1473 + 8 * a * (298 + 25 * (17 - 9 * a) * a)))) * b
                ) * c**2 - 210 * ( - 1 + a) * (
                    + 5 * a**2 * (b + a * ( - 4 + 3 * b)) 
                    - 2 * a * (2 * b + a * ( - 5 + 6 * b + 3 * a * ( - 5 + 4 * b))) * c 
                    + ( - 2 * a * (1 + 3 * a + 6 * a**2) + b + a * (3 + 2 * a * (3 + 5 * a)) * b) * c**2
                ) * numpy.log(1 - a)
            )
        )

    coefficient = chi2**3 * power2 * (1 + redshift2)**2 * formula
    
    return coefficient


In [6]:
INTEGRAL = integral1(CHI1, CHI2, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient1(CHI1, CHI2, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(11003.57056006051, 11003.676353273713, 9.614444022920665e-06)